In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
#label encoder
from sklearn.preprocessing import LabelEncoder
def dummyEncode(df):
        columnsToEncode = list(df.select_dtypes(include=['category','object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            
                df[feature] = le.fit_transform(df[feature])
           
        return df
#splitter
def split(X,y,splitratio):
    trainsize=int(X.shape[0]*splitratio)
    xtrain=X[:trainsize]
    ytrain=y[:trainsize]
    xtest=X[trainsize:]
    ytest=y[trainsize:]
    return xtrain,ytrain,xtest,ytest
#get accuracy
def getacc(y_pred,yp2):
    right=0
    for i in range(len(y_pred)):
        if(y_pred[i]==yp2[i]):
            right+=1
    return (100*right/float(len(y_pred)))

In [3]:
ds = pd.read_csv('train.csv')
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 15 columns):
ID                                      600000 non-null int64
age                                     600000 non-null float64
sex                                     600000 non-null int64
chest                                   600000 non-null float64
resting_blood_pressure                  600000 non-null float64
serum_cholestoral                       600000 non-null float64
fasting_blood_sugar                     600000 non-null int64
resting_electrocardiographic_results    600000 non-null int64
maximum_heart_rate_achieved             600000 non-null float64
exercise_induced_angina                 600000 non-null int64
oldpeak                                 600000 non-null float64
slope                                   600000 non-null int64
number_of_major_vessels                 600000 non-null int64
thal                                    600000 non-null int64
class    

In [4]:
ds.head()

,ID,age,sex,chest,resting_blood_pressure,serum_cholestoral,fasting_blood_sugar,resting_electrocardiographic_results,maximum_heart_rate_achieved,exercise_induced_angina,oldpeak,slope,number_of_major_vessels,thal,class
0,0,49.207124,0,4.000000,162.996167,181.108682,0,0,148.227858,1,0.944547,2,0,3,1
1,1,53.628425,1,1.741596,130.233730,276.474630,0,2,152.917139,0,0.119070,2,0,3,0
2,2,49.591426,1,4.000000,146.999012,223.300517,1,2,102.352090,1,1.616747,2,2,7,1
3,3,58.991445,1,4.000000,112.369143,187.245501,0,0,158.164750,1,0.000000,1,1,7,1
4,4,51.053602,1,1.954609,138.032047,238.482868,0,0,172.540828,0,1.150464,1,1,3,0


In [5]:
#drop features, make X,y
X=ds

X=X.drop(['ID'],axis=1) #drop features
X=X.dropna()
y=X['class']
X=X.drop(['class'],axis=1)#drop the class to be predicted
print (X.shape)

(600000, 13)


In [6]:
X=dummyEncode(X)

In [7]:
xtrain,ytrain,xtest,ytest=split(X,y,0.8)
#convert to list, coz idk how to access a fckng dataframe
yp2=[]
for i in ytest:
    yp2.append(i)

In [8]:
#naive bayes
def naivebayes(X,y,xtest):
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    y_pred = gnb.fit(xtrain, ytrain).predict(xtest)
    print("naive bayes")
    #print (getacc(y_pred,yp2))
    return y_pred


In [9]:
#Decision Tree Classifier , run multiple times to get diff accuracy 
def dectree(X,y,xtest):
    from sklearn import tree
    clf = tree.DecisionTreeClassifier(max_depth=1)
    clf = clf.fit(xtrain, ytrain)
    y_pred=clf.predict(xtest)
    print("dec-tree")
    return y_pred

In [10]:
#adabooster , n estimators
def adabooster(X,y,xtest):
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.tree import DecisionTreeClassifier
    bdt = AdaBoostClassifier(
                             algorithm="SAMME.R",
                             n_estimators=100,learning_rate=0.6)
    bdt.fit(xtrain,ytrain)
    y_pred=bdt.predict(xtest)
    print("adabooster")
    #print(getacc(y_pred,yp2))
    return y_pred



In [11]:
#Random forest classifier
def randomforest(X,y,xtest):
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(n_estimators=100,random_state=0)
    clf.fit(X, y)
    y_pred=clf.predict(xtest)
    print("randome forest")
    return y_pred


In [12]:
#Gradient Boosting Classifier
def gradboost(X,y,xtest):
    from sklearn.ensemble import GradientBoostingClassifier
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=2,max_depth=5,random_state=0)
    clf.fit(X, y)
    y_pred=clf.predict(xtest)
    print("Gradient boost")
    print(getacc(y_pred,yp2))
    return getacc(y_pred,yp2)
def gradboost1(X,y,xtest):
    from sklearn.ensemble import GradientBoostingClassifier
    clf = GradientBoostingClassifier(n_estimators=500,learning_rate=0.6, max_depth=1,random_state=0)
    clf.fit(X, y)
    y_pred=clf.predict(xtest)
    print("Gradient boost")
    #print(getacc(y_pred,yp2))
    return y_pred

In [13]:
def callall(X,y,xtest):
    gradboost(X,y,xtest)
    randomforest(X,y,xtest)
    naivebayes(X,y,xtest)
    dectree(X,y,xtest)
    #adabooster(X,y,xtest)
def callall2(X,y,xtest):
    #y1=gradboost1(X,y,xtest)
    #y2=dectree(X,y,xtest)
    #y3=randomforest(X,y,xtest)
    y4=adabooster(X,y,xtest)
    #y5=naivebayes(X,y,xtest)
    yf=[]
    for i in range(len(y4)):
        x=0
        #if(y1[i]):
        #    x+=1
        #if(y2[i]):
        #    x+=0
        #if(y3[i]):
        #    x+=1
        if(y4[i]):
            x+=1
        #if(y5[i]):
        #    x+=1
        if(x>0):
            yf.append(0)
        else:
            yf.append(1)
    return yf
            
    


In [14]:
td=pd.read_csv('test.csv')

td=dummyEncode(td)
Srno=[]
for i in td['ID']:
    Srno.append(i)
td=td.drop(['ID'],axis=1)
td=dummyEncode(td)

In [15]:

#yfinal1=callall2(X,y,td)


In [16]:
from sklearn.neighbors import KNeighborsClassifier
def get_acc2(kx):
    #TODO
    right=0;
    total=xtest.shape[0]
    for i in range(total):
        neigh = KNeighborsClassifier(n_neighbors=kx)
        neigh.fit(xtrain, ytrain)
        print(xtest[i])
        #group=neigh.predict([xtest[i]])
        #if(group==ytest[i]):
         #   right+=1
    return 100*(right/float(total))

In [29]:
#get_acc2(7)
for i in range(20):
    if(i==0):
        continue
    neigh = KNeighborsClassifier(n_neighbors=i)
    neigh.fit(X, y)
    group=neigh.predict(xtest)
    print(i," ",getacc(group,yp2))


1   100.0
2   88.4875
3   89.5575
4   86.85666666666667
5   87.27416666666667
6   85.84416666666667
7   86.19333333333333
8   85.3075
9   85.40083333333334
10   84.79
11   84.92666666666666
12   84.44166666666666
13   84.52416666666667
14   84.21166666666667
15   84.17083333333333
16   83.90833333333333
17   83.95083333333334
18   83.61333333333333
19   83.71416666666667


In [38]:
neigh = KNeighborsClassifier(n_neighbors=7)
neigh.fit(X, y)
group=neigh.predict(td)
df3 = pd.DataFrame()
print(len(group))
print(len(ytest))
print(group)



400000
120000
[1 1 0 ... 0 0 1]


In [39]:
df3['ID'] = Srno

In [40]:
df3['class'] = group

In [41]:
df3.to_csv('Sub.csv',index=False)